![NYC Skyline](nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

In [176]:
# Import necessary packages
import pandas as pd
import numpy as np
import os

# Begin coding here ...
# Use as many cells as you like

cwd = os.getcwd()
print(cwd)

files = os.listdir(f'{cwd}/data')
print(files)

/work/files/workspace
['airbnb_room_type.xlsx', 'airbnb_price.csv', 'airbnb_last_review.tsv']


In [177]:
price_df = pd.read_csv("data/airbnb_price.csv")

print(price_df.info())
print(price_df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   price        25209 non-null  object
 2   nbhood_full  25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB
None
listing_id     0
price          0
nbhood_full    0
dtype: int64


In [178]:
room_xls = pd.ExcelFile("data/airbnb_room_type.xlsx")
room_df = room_xls.parse(0)

room_df.info()
print(room_df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   description  25199 non-null  object
 2   room_type    25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB
listing_id      0
description    10
room_type       0
dtype: int64


In [179]:
review_df = pd.read_csv("data/airbnb_last_review.tsv", sep='\t')

review_df.info()
review_df = review_df.dropna()
print(review_df.isna().sum())

times_formatted = pd.to_datetime(review_df['last_review'], infer_datetime_format=True, errors='coerce', format='%B %d %Y')
review_df['last_review'] = times_formatted
review_df = review_df.sort_values(by='last_review').reset_index(drop=True)

print(review_df.head())

oldest_review = review_df['last_review'].min()
newest_review = review_df['last_review'].max()
print(oldest_review, newest_review)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   host_name    25201 non-null  object
 2   last_review  25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB
listing_id     0
host_name      0
last_review    0
dtype: int64
   listing_id host_name last_review
0    29178331       Fon  2019-01-01
1    15840089   Yolanda  2019-01-01
2      917058     Darla  2019-01-01
3     4365765     Marie  2019-01-01
4    24075850     Cindy  2019-01-01
2019-01-01 00:00:00 2019-07-09 00:00:00


In [180]:
room_df['room_type'] = room_df['room_type'].str.upper()

private_room_count = room_df['room_type'].value_counts().loc['PRIVATE ROOM']
print(private_room_count)


11356


In [181]:
price_df['price'] = price_df['price'].str.strip(' dollars').astype(int)
price_df.dropna(subset=['price'])

price_df['price'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 25209 entries, 0 to 25208
Series name: price
Non-Null Count  Dtype
--------------  -----
25209 non-null  int64
dtypes: int64(1)
memory usage: 197.1 KB


In [182]:
print(price_df.shape)
price_df = price_df[price_df['price'] != 0]
print(price_df.shape)

mean_price = price_df['price'].describe().loc['mean']
mean_price = round(mean_price, 2)
print(mean_price)

cols = ['first_reviewed', 'last_reviewed', 'nb_private_rooms', 'avg_price']
vals = [oldest_review, newest_review, private_room_count, mean_price]

review_dates_dict = dict(zip(cols, vals))

review_dates = pd.DataFrame(data=review_dates_dict, index=[0])
review_dates

(25209, 3)
(25202, 3)
141.82


,first_reviewed,last_reviewed,nb_private_rooms,avg_price
0,2019-01-01,2019-07-09,11356,141.82
